In [1]:
!pip install datasets transformers Sentencepiece nltk

In [2]:
# download hugging face datasets

import datasets
from datasets import concatenate_datasets

import sentencepiece
from nltk.translate.bleu_score import corpus_bleu
from tqdm import tqdm
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)




2024-03-31 14:54:39.364642: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 14:54:39.364746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 14:54:39.507654: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


cuda


In [3]:
train_Data = datasets.load_dataset("wmt16","de-en", split="train[:50000]")
val_Data = datasets.load_dataset("wmt16","de-en", split="validation")
test_Data = datasets.load_dataset("wmt16","de-en", split="test")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/9e0038fe4cc117bd474d2774032cc133e355146ed0a47021b2040ca9db4645c0. Subsequent calls will reuse this data.


## Load the model

In [4]:
model_name = "google-t5/t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
translations_validation = []
translations_testing = []
val_test_Data = concatenate_datasets([val_Data, test_Data])
# testing_data = test_Data
print(device)
model.to(device)

# for data in [validation_data, testing_data]:
for example in tqdm(val_test_Data):
    # Prepend the prefix for translation task
    input_text = "translate English to German: " + example["translation"]["en"]  # Modified here
    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    # Generate translation
    translated_ids = model.generate(input_ids,  max_length=512, num_beams=4, early_stopping=True).to(device)
    # Decode the translated ids
    translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    # if data == validation_data:
    translations_validation.append(translated_text)
    # else:
        # translations_testing.append(translated_text)

# Calculate BLEU score for validation and testing translations
reference_validation = [example["translation"]["de"] for example in val_test_Data]  # Modified here
# reference_testing = [example["translation"]["de"] for example in testing_data]

bleu_score_validation = corpus_bleu([[ref] for ref in reference_validation], translations_validation)
# bleu_score_testing = corpus_bleu([[ref] for ref in reference_testing], translations_testing)

print("BLEU score for validation set:", bleu_score_validation)
# print("BLEU score for testing set:", bleu_score_testing)


cuda


100%|██████████| 5168/5168 [28:55<00:00,  2.98it/s]


BLEU score for validation set: 0.6863329636040714


In [11]:
meteor = datasets.load_metric("meteor")
meteor_score = meteor.compute(predictions=translations_validation, references=reference_validation)
print("METEOR score for validation set:", meteor_score)
# install nltk


METEOR score for validation set: {'meteor': 0.5742548079549654}


In [7]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00


In [8]:
from bert_score import score
P, R, F1 = score(translations_validation, reference_validation, lang="en", verbose=True)
print("BERT Score for validation set:", F1.mean().item())

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/160 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/81 [00:00<?, ?it/s]

done in 55.43 seconds, 93.23 sentences/sec
BERT Score for validation set: 0.9165949821472168


In [9]:
# bleu 1 score
def compute_bleu():
    w1 = [1, 0, 0, 0]
    w2 = [0.5, 0.5, 0, 0]
    w3 = [1/3, 1/3, 1/3, 0]
    w4 = [0.25, 0.25, 0.25, 0.25]
    bleu1 = corpus_bleu([[ref] for ref in reference_validation], translations_validation, weights=w1)
    bleu2 = corpus_bleu([[ref] for ref in reference_validation], translations_validation, weights=w2)
    bleu3 = corpus_bleu([[ref] for ref in reference_validation], translations_validation, weights=w3)
    bleu4 = corpus_bleu([[ref] for ref in reference_validation], translations_validation, weights=w4)
    print("BLEU-1 score for validation set:", bleu1)
    print("BLEU-2 score for validation set:", bleu2)
    print("BLEU-3 score for validation set:", bleu3)
    print("BLEU-4 score for validation set:", bleu4)

compute_bleu()

BLEU-1 score for validation set: 0.8693316903269345
BLEU-2 score for validation set: 0.7955778532875366
BLEU-3 score for validation set: 0.7345549424282534
BLEU-4 score for validation set: 0.6863329636040714
